#Установка зависимостей
### Dont care просто выполни 13 скрытых ячеек

In [29]:
pip install transformers==2.2.2


In [30]:
pip install neuralcoref

In [31]:
pip install spacy==2.1.3

In [32]:
!sudo apt-get install swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [33]:
pip install jamspell

In [34]:
!git clone https://github.com/vlomme/Bert-Russian-punctuation

fatal: destination path 'Bert-Russian-punctuation' already exists and is not an empty directory.


In [35]:
cd Bert-Russian-punctuation/

/content/Bert-Russian-punctuation


In [36]:
pip install pymorphy2

In [37]:
pip install pytorch-pretrained-bert

In [38]:
!gdown https://drive.google.com/uc?id=190dLqhRjqgNJLKBqz0OxQ3TzxSm5Qbfx

Downloading...
From: https://drive.google.com/uc?id=190dLqhRjqgNJLKBqz0OxQ3TzxSm5Qbfx
To: /content/Bert-Russian-punctuation/bert_punctuation.tar.gz
660MB [00:04, 134MB/s]


In [39]:
!wget https://github.com/bakwc/JamSpell-models/raw/master/ru.tar.gz

--2020-12-29 09:42:22--  https://github.com/bakwc/JamSpell-models/raw/master/ru.tar.gz
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bakwc/JamSpell-models/master/ru.tar.gz [following]
--2020-12-29 09:42:23--  https://raw.githubusercontent.com/bakwc/JamSpell-models/master/ru.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39663667 (38M) [application/octet-stream]
Saving to: ‘ru.tar.gz.1’

ru.tar.gz.1         100%[===================>]  37.83M   161MB/s    in 0.2s    

2020-12-29 09:42:24 (161 MB/s) - ‘ru.tar.gz.1’ saved [39663667/39663667]



In [40]:
!tar -xf /content/Bert-Russian-punctuation/ru.tar.gz

In [41]:
pip install python-pptx

In [66]:
pip install python-pptx-interface

     |████████████████████████████████| 2.0MB 5.2MB/s 


#Основной код





##Презантация в Таблицу

In [14]:
from google.colab import files

filename = list(files.upload().keys())[0]

Saving 20201130_Статус_AI_БСР_final (2).pptx to 20201130_Статус_AI_БСР_final (2).pptx


In [122]:
import pandas as pd
d = {}


def summ_( shapes):
    """
    На входе словарь {match: replacement, ... } и заменяет все совпадения графики и таблицы не готовы
    """

    for shape in shapes:
        
        if shape.has_text_frame:
            
            text_frame = shape.text_frame
            text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE
            for paragraph in text_frame.paragraphs:
                     #   print('new par agraph')
                      #print(paragraph.runs)
                      counter = 0
                      runs = ''
                      for run in paragraph.runs:
                            
                            d.update({str(run.text) :'None'})
                    
summ_(shapes)
#df.head()
df = pd.DataFrame([d])
df.head()

,2,7,tttt,Статус,AI,Статус AI-трансформации,Блока Стратегия и Развитие,Миссия,AI,в БСР: что мы даем,Сберу,"«Второе мнение» при принятии управленческих решений, основанное на интерпретированных математических методах:",Новые,AI-driven,продукты для принятия управленческих решений,Дополнительные драйверы и гипотезы для управления эффективностью и численностью,Интеллектуальная оптимизация и реинжиниринг клиентских путей и процессов,Потенциально: прогнозирование векторов стратегии,Сбера,с помощью,Текущий эффект от внедрения,инициатив в БСР,Общий,shadow*-,эффект от,в БСР,*,"- пока эффект от реализации модели численности. Не входит в бизнес-план БСР, но учтен в планах блоков на 2021 год",202,1,Кол-во ПШЕ,оптимизация (,shadow),Экономия бюджета,(,млрд.руб,",","1,5","2,8",,...,?,Добавить в,СберИдею,"модуль, который бы позволял пользователям добавлять свою идею и предлагать идею на голосование","Создать мобильное приложение, в котором можно будет добавлять свою идею и голосовать.",СберИдее,"указывается основная идея, которой можно предложить варианты развития проекта, в том числе описание выгоды, которую получит пользователь, рекомендации по использованию. Приложение содержит графические подсказки по использованию идеи. Приложение может быть интегрировано в портал государственных услуг",Что «думает»,наш,о том как сделать всех людей счастливыми?,Как сделать всех людей счастливыми?,"Начать нужно с себя, с собственных убеждений и привычек. Верить в себя, в свои мечты, в свои планы и цели. Уделять внимание своему развитию, самосовершенствованию. И быть в гармонии с самим собой!",Все люди (даже самые несчастные) хотят счастья.,"Если человек хочет счастья, значит у него есть причины, чтобы хотеть. Например, это может быть желание отомстить. Или желание получить что-то ценное. Или просто желание быть счастливым.","На самом деле, для того, чтобы быть счастливым, совсем не обязательно кого-то хотеть. Для этого просто достаточно быть в согласии со своим внутренним ""Я"". Не важно, что именно, главное, чтобы это было гармоничным и естественным.","И еще важно, что для того, чтобы быть счастливым, вовсе не обязательно делать что-то, что нарушает общепринятые нормы морали.","Главное, что счастье человека должно быть взаимным!",Счастье в наших руках!,Как ты можешь сделать всех людей счастливыми?,1),С помощью создания искусственного разума:,"а) ""Запрограммировать"" всех на безусловное, ничем не обусловленное принятие любых решений","б) Привлечь к решению этой задачи специалистов по генной инженерии, биоинженерии,",роботехнике,в) Задать каждому человеку только одну цель: быть счастливым,г) Задавать вопросы всем без исключения людям: «Как тебе нравится моя жизнь сейчас?»,2) С помощью системы образования:,"а) Привлечь к решению этой задачи лучших преподавателей школ, ВУЗов, специалистов по социальной психологии, нейробиологии.","б) Создать в школах, ВУЗах, компаниях специальные классы, где обучающиеся, с помощью специальных тестов, смогут проверить свои возможности.","в) Создать специальное учебное заведение, в котором преподаватели ВУЗов и школ могли бы проводить в рамках учебных программ практические семинары и мастер-классы по этим вопросам.",г) Обеспечить каждому возможность бесплатного обучения и получения дополнительного образования.,д) Обеспечить каждому возможность бесплатно проходить курсы повышения квалификации по интересующим его вопросам.,"ж) Создать в Интернете сайт, на котором любой человек мог бы прочитать и получить дополнительную информацию о том, что такое счастье.",@eight.ai –,"мой инстаграм, видео и тексты для которого пишет нейронная сеть",Сферы применения,ru:GPT-3,"обширны, мы можем генерировать ответы на обратную связь, писать поздравления, презентации, тексты на заданные темы по шаблону и без, писать музыку, делать новый дизайн с помощью текстового описания и",.,"Технология уже доступна нам, и в нашей команде есть необходимые специалисты для работы с ней."
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None

In [18]:
cd /content

/content


In [19]:
df.to_excel("output.xlsx")

##Таблица в презентацию

In [115]:
edit = list(files.upload().keys())[0]

Saving output (2).xlsx to output (2).xlsx


In [ ]:
import pandas as pd
inp = pd.read_excel(edit)
replac = inp.to_dict()
inp.head()

In [117]:
cheker = 0
for match, replacement in replac.items():
  if replacement[0]!='None':
    cheker+=1
print('total changeS: ',cheker)

total changeS:  3


In [119]:
from pptx import Presentation
import pptx
import copy
from pptx.enum.shapes import MSO_SHAPE
from pptx.enum.text import MSO_AUTO_SIZE

prs = Presentation(filename)
slides = [slide for slide in prs.slides]
shapes = []
for slide in slides:
    for shape in slide.shapes:
        shapes.append(shape)
if slides!=None and shapes!=None:
  print("Read corect")
else:
  print("Did you lost some info?")

Read corect


#Баг в ячейке снизу, смысл в том что в функцию replace_text подается словарь с заменой и список обьектов с текстом, 

In [121]:

import jamspell
from bert_punctuation import Bert_punctuation

Bert_punctuation = Bert_punctuation()
corrector = jamspell.TSpellCorrector()
model_path = '/content/Bert-Russian-punctuation/ru_small.bin'
corrector.LoadLangModel(model_path)

def grammar(inp):
  
  l = list()
  l+=[corrector.FixFragment(inp)]
  
  return Bert_punctuation.predict(l)[0]

def duplicate_slide(pres, index):
    """
    pres - обьект презентации
    индес - номер слайда
    """
    template = pres.slides[index]
    try:
        blank_slide_layout = pres.slide_layouts[12]
    except:
        blank_slide_layout = pres.slide_layouts[len(pres.slide_layouts)]

    copied_slide = pres.slides.add_slide(blank_slide_layout)

    for shp in template.shapes:
        el = shp.element
        newel = copy.deepcopy(el)
        copied_slide.shapes._spTree.insert_element_before(newel, 'p:extLst')

    for _, value in six.iteritems(template.part.rels):
        # проверка существованияз
        if "notesSlide" not in value.reltype:
            copied_slide.part.rels.add_relationship(
                value.reltype,
                value._target,
                value.rId
            )

    return copied_slide




#def duplicate_slide(prs, 2)
def replace_text(replacements, shapes):
    """
    На входе словарь {'match': 'replacement', ... } и заменяет все совпадения графики и таблицы не готовы
    """

    for shape in shapes:
    
        for match, replacement in replacements.items():
           
            if shape.has_text_frame:
               
                if (shape.text.find(match)) != -1 and  replacement[0]!='None':
                    
                    
                    text_frame = shape.text_frame
                    text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE
                    
                    
                    for paragraph in text_frame.paragraphs:
                     
                    
                      for run in paragraph.runs:
                           
                           new_text = runs.replace(str(match), str(replacement[0]))
                           run.text = new_text
                      

 
replace_text(replac, shapes) 


None
None
None
27 ноября 2020 d
558800
558800
558800
Статус AI-трансформации d
508000
Блока Стратегия и Развитие d


In [113]:
prs.save('/content/test15.pptx')

In [92]:
def replace_text(replacements, shapes):
    """Takes dict of {match: replacement, ... } and replaces all matches.
    Currently not implemented for charts or graphics.
    """
    for shape in shapes:
        for match, replacement in replacements.items():
            if shape.has_text_frame:
                if (shape.text.find(match)) != -1:
                    text_frame = shape.text_frame
                    for paragraph in text_frame.paragraphs:
                        for run in paragraph.runs:
                            cur_text = run.text
                            new_text = cur_text.replace(str(match), str(replacement))
                            run.text = 'fg'
                            print( run.text , new_text)
                            break
            if shape.has_table:
                for row in shape.table.rows:
                    for cell in row.cells:
                        if match in cell.text:
                            new_text = cell.text.replace(match, replacement)
                            cell.text = new_text

replace_text({'27 ноября 2020': 'replacement '}, shapes) 

fg 2


In [93]:
prs.save('/content/test11.pptx')

#dont open, unuseful

In [ ]:
def get_slide_count(prs):

    slidecount = 0
    for slide in prs.slides:
        slidecount += 1
    return slidecount


def delete_slide(prs, slide):
    """ Delete a slide out of a powerpoint presentation"""
    id_dict = { slide.id: [i, slide.rId] for i,slide in enumerate(prs.slides._sldIdLst) }
    slide_id = slide.slide_id
    prs.part.drop_rel(id_dict[slide_id][1])
    del prs.slides._sldIdLst[id_dict[slide_id][0]]


def get_single_slide_pres(prs, slidetokeep):
    for idx, slide in enumerate(prs.slides):
        if idx < slidetokeep:
            delete_slide(prs, slide)
        elif (idx > slidetokeep):
            delete_slide(prs, slide)
    prs.save(str(slidetokeep + 1) + ".pptx")


pptxfilepath = "C://Users//alexd//Downloads//ШаблонНС.pptx"
prs = Presentation(pptxfilepath)
slidecount = get_slide_count(prs)
for i in range(slidecount):
    prs_backup = Presentation(pptxfilepath)
    get_single_slide_pres(prs_backup, i)
    prs_backup = None
prs.

In [ ]:
from pptx import Presentation
from pptx.util import Inches
from pptx.enum.text import MSO_AUTO_SIZE
from pptx.enum.text import MSO_ANCHOR

def words_counter(str):
    return len(str.split())

text= "тест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест текст"
text= text+"тест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест текст"+"тест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест текст"
text= text+"тест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест текст"+"тест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест текст"
text= text+"тест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест текст"+"тест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест тексттест текст"


prs = Presentation()
titleLayout = prs.slide_layouts[0]
slide = prs.slides.add_slide(titleLayout)
shape = slide.shapes.add_textbox(Inches(1), Inches(1.1),Inches(2), Inches(1.8))
#font.size = Pt(14)
#text_frame.fit_text(font_family=u'Calibri', max_size=18, bold=False, italic=False, font_file=None)

text_frame = shape.text_frame
#print(shape.text_frame.auto_size)
text_frame.text = text
text_frame.fit_text()
shape.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE

#text_frame.margin_bottom = Inches(0.08)
#text_frame.margin_left = 0
#text_frame.vertical_anchor = MSO_ANCHOR.LEFT
text_frame.word_wrap = True



#shape.text_frame.auto_size = MSO
#textFrame.fit_text('calibriz.ttf', max_size=18)
#textFrame.auto_size, textFrame.word_wrap

tf = shape.text_frame



tf.text = text

textFrame = shape.text_frame



textParagraph = textFrame.add_paragraph()










prs.add_slide()
title_only_slide_layout = prs.slide_layouts[5]
slide = prs.slides.add_slide(title_only_slide_layout)
shapes = slide.shapes

shapes.title.text = 'Adding a Table'

rows = cols = 2
left = top = Inches(2.0)
width = Inches(6.0)
height = Inches(0.8)

table = shapes.add_table(rows, cols, left, top, width, height).table

# set column widths
table.columns[0].width = Inches(2.0)
table.columns[1].width = Inches(4.0)

# write column headings
table.cell(0, 0).text = 'Foo'
table.cell(0, 1).text = 'Bar'

# write body cells
table.cell(1, 0).text = 'Baz'
table.cell(1, 1).text = 'Qux'


prs.add_slide()
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)
img_path = 'C://Users//alexd//Desktop//photo_2020-08-12_07-20-54.jpg'
left = top = Inches(1)
pic = slide.shapes.add_picture(img_path, left, top)

left = Inches(5)
height = Inches(5.5)
pic = slide.shapes.add_picture(img_path, left, top, height=height)

textParagraph.text = text








prs.save("PptxWordWrapTest.pptx")